In [1]:
# Load Library

packages <- c("dplyr", "haven","ggplot2","MASS", "randomForest", "caret", "ranger", "parallel")

install_if_missing <- function(pkg) {
  if (!requireNamespace(pkg, quietly = TRUE)) {
    install.packages(pkg)
  }
  library(pkg, character.only = TRUE)
}

lapply(packages, install_if_missing)

select <- dplyr::select


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'MASS'


The following object is masked from 'package:dplyr':

    select


randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:ggplot2':

    margin


The following object is masked from 'package:dplyr':

    combine


Loading required package: lattice


Attaching package: 'ranger'


The following object is masked from 'package:randomForest':

    importance




[[1]]
[1] "dplyr"     "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     

[[2]]
[1] "haven"     "dplyr"     "stats"     "graphics"  "grDevices" "utils"    
[7] "datasets"  "methods"   "base"     

[[3]]
 [1] "ggplot2"   "haven"     "dplyr"     "stats"     "graphics"  "grDevices"
 [7] "utils"     "datasets"  "methods"   "base"     

[[4]]
 [1] "MASS"      "ggplot2"   "haven"     "dplyr"     "stats"     "graphics" 
 [7] "grDevices" "utils"     "datasets"  "methods"   "base"     

[[5]]
 [1] "randomForest" "MASS"         "ggplot2"      "haven"        "dplyr"       
 [6] "stats"        "graphics"     "grDevices"    "utils"        "datasets"    
[11] "methods"      "base"        

[[6]]
 [1] "caret"        "lattice"      "randomForest" "MASS"         "ggplot2"     
 [6] "haven"        "dplyr"        "stats"        "graphics"     "grDevices"   
[11] "utils"        "datasets"     "methods"      "base"        

[[7]]
 [1] "ranger"       "caret"        "lattice"      "randomForest" "MASS"        
 [6] "ggplot2"      "haven"        "dplyr"        "stats"        "graphics"    
[11] "grDevices"    "utils"        "datasets"     "methods"      "base"        

[[8]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"

# Maching Learning

In [2]:
df1 <- read.csv("Data/merge_df_cln.csv")
df1 <- df1 %>% 
        mutate(hyper = ifelse((bp_sys_mean >= 140 | bp_dia_mean >= 90), 1 , 0)) %>% 
        filter(htn_aware == "No") %>% #filter with Aware
        dplyr::select(-"X", -"bp_sys_mean", -"svy_id", -"bp_dia_mean", -"htn_aware")

colnames(df1)
# df1$hyper <- as.factor(df1$hyper)
# summary(df1$hyper)

[1] "svy_year"        "demo_age_cat"    "demo_race"       "demo_race_black"
 [5] "demo_age_years"  "demo_pregnant"   "demo_gender"     "cc_bmi"         
 [9] "DRXTPROT"        "DRXTCARB"        "DRXTTFAT"        "DRXTSFAT"       
[13] "DRXTMFAT"        "DRXTPFAT"        "DRXTCHOL"        "DRXTFIBE"       
[17] "DRXTVB6"         "DRXTVB12"        "DRXTFOLA"        "DRXTVC"         
[21] "DRXTCALC"        "DRXTPHOS"        "DRXTMAGN"        "DRXTIRON"       
[25] "DRXTZINC"        "DRXTCOPP"        "DRDTSODI"        "DRXTPOTA"       
[29] "DRXTSELE"        "DRXTCAFF"        "DRXTTHEO"        "DRXTALCO"       
[33] "DRXTMOIS"        "DRXTS040"        "DRXTS060"        "DRXTS080"       
[37] "DRXTS100"        "DRXTS120"        "DRXTS140"        "DRXTS160"       
[41] "DRXTS180"        "DRXTM161"        "DRXTM181"        "DRXTM201"       
[45] "DRXTM221"        "DRXTP182"        "DRXTP183"        "DRXTP184"       
[49] "DRXTP204"        "DRXTP205"        "DRXTP225"        "DRXTP226"       
[53] "DRD320GW"        "hyper"

## Random Forest

### Desrease size of majority

In [73]:
table(df1$hyper)

# Separate majority and minority classes
df1_majority <- df1 %>% filter(hyper == 0)
df1_minority <- df1 %>% filter(hyper == 1)

set.seed(42)
# Oversample minority class (duplicate existing data)
df1_majority_oversampled <- df1_majority %>% sample_n(nrow(df1_minority), replace = F)

# Combine balanced dataset
df1_balanced_decrease <- bind_rows(df1_minority, df1_majority_oversampled)

# Check new class distribution
table(df1_balanced_decrease$hyper)


    0     1 
29845  3680 


   0    1 
3680 3680 

In [64]:
# Change here for different resample method
df <- df1_balanced_decrease


# Split into training (70%) and testing (30%) sets
set.seed(42)
train_index <- createDataPartition(df$hyper, p = 0.7, list = FALSE)
train_data <- df[train_index, ]
test_data <- df[-train_index, ]

train_data$hyper <- as.factor(train_data$hyper)
test_data$hyper <- as.factor(test_data$hyper)

In [65]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data, 
                   num.trees = 500, 
                   mtry = sqrt(ncol(train_data) - 1), 
                   importance = "impurity",
                   probability = TRUE,  # Enables probability prediction
                   num.threads = num_cores)

# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# Convert probabilities to class labels
predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# Model Evaluation
conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# Print Accuracy, Sensitivity, Specificity
print(conf_matrix)

# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 754 253
         1 350 851
                                          
               Accuracy : 0.7269          
                 95% CI : (0.7078, 0.7454)
    No Information Rate : 0.5             
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.4538          
                                          
 Mcnemar's Test P-Value : 9.252e-05       
                                          
            Sensitivity : 0.6830          
            Specificity : 0.7708          
         Pos Pred Value : 0.7488          
         Neg Pred Value : 0.7086          
             Prevalence : 0.5000          
         Detection Rate : 0.3415          
   Detection Prevalence : 0.4561          
      Balanced Accuracy : 0.7269          
                                          
       'Positive' Class : 0               
         

demo_age_years    demo_age_cat        DRXTMOIS        DRXTCARB        DRXTCALC 
     351.108904      240.962979       49.696176       43.719252       39.300271 
         DRXTVC        DRDTSODI        DRXTCAFF         DRXTVB6        DRXTPOTA 
      39.223848       38.839815       38.618285       38.537601       38.515220 
       DRXTFOLA        DRXTM201        DRXTCHOL        DRXTVB12        DRXTPROT 
      38.267965       38.192021       37.378188       37.225468       36.854727 
       DRXTS120        DRXTP204        DRXTIRON        DRXTSELE        DRXTZINC 
      36.662841       36.598173       36.270201       36.147135       36.105218 
       DRXTFIBE        DRXTP183        DRXTCOPP        DRXTPHOS        DRXTM161 
      35.723475       35.480631       35.416426       35.333891       35.043836 
       DRXTP182        DRXTM181        DRD320GW        DRXTPFAT        DRXTS140 
      35.005980       34.804971       34.663185       34.545867       34.258294 
       DRXTMAGN        DRXTMFAT          cc_bmi        DRXTS180        DRXTTFAT 
      34.040644       33.825261       33.200892       32.975236       32.927762 
       DRXTS080        DRXTS160        DRXTS040        DRXTSFAT        DRXTS100 
      32.707109       32.230404       31.736682       31.581750       31.065319 
       DRXTS060        DRXTM221        DRXTP226        DRXTP225        DRXTP205 
      29.292366       28.672196       26.963092       25.802190       24.353936 
       DRXTTHEO        svy_year        DRXTALCO     demo_gender        DRXTP184 
      23.466477       22.300750       20.996893       17.276316       15.741951 
      demo_race demo_race_black   demo_pregnant 
      12.744057        8.226591        0.000000

### Increase size of minority (Bootstrap)

In [39]:
table(df1$hyper)

# Separate majority and minority classes
df1_majority <- df1 %>% filter(hyper == 0)
df1_minority <- df1 %>% filter(hyper == 1)

set.seed(24)
# Oversample minority class (duplicate existing data)
df1_minority_oversampled <- df1_minority %>% sample_n(nrow(df1_majority), replace = T)

# Combine balanced dataset
df1_balanced_bootstrap <- bind_rows(df1_majority, df1_minority_oversampled)

# Check new class distribution
table(df1_balanced_bootstrap$hyper)


    0     1 
29845  3680 


    0     1 
29845 29845 

In [40]:
# Change here for different resample method
df <- df1_balanced_bootstrap


# Split into training (70%) and testing (30%) sets
set.seed(42)
train_index <- createDataPartition(df$hyper, p = 0.7, list = FALSE)
train_data <- df[train_index, ]
test_data <- df[-train_index, ]

train_data$hyper <- as.factor(train_data$hyper)
test_data$hyper <- as.factor(test_data$hyper)

In [41]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data, 
                   num.trees = 500, 
                   mtry = sqrt(ncol(train_data) - 1), 
                   importance = "impurity",
                   probability = TRUE,  # Enables probability prediction
                   num.threads = num_cores)

# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# Convert probabilities to class labels
predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# Model Evaluation
conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# Print Accuracy, Sensitivity, Specificity
print(conf_matrix)

# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 5777   19
         1  192 5950
                                          
               Accuracy : 0.9823          
                 95% CI : (0.9798, 0.9846)
    No Information Rate : 0.5             
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.9647          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.9678          
            Specificity : 0.9968          
         Pos Pred Value : 0.9967          
         Neg Pred Value : 0.9687          
             Prevalence : 0.5000          
         Detection Rate : 0.4839          
   Detection Prevalence : 0.4855          
      Balanced Accuracy : 0.9823          
                                          
       'Positive' Class : 0               
   

demo_age_years    demo_age_cat        DRXTMOIS        DRXTCARB        DRDTSODI 
      3360.9946       2366.7521        521.7803        462.8543        436.2497 
         DRXTVC        DRXTCAFF        DRXTM201        DRXTFOLA         DRXTVB6 
       425.6430        420.4418        419.1538        418.2492        410.9722 
       DRXTIRON        DRXTSELE        DRXTM161        DRXTCALC        DRXTVB12 
       404.4619        401.8032        398.3815        396.3730        395.8736 
       DRXTCOPP        DRXTFIBE        DRXTCHOL        DRXTPOTA        DRXTPROT 
       394.6566        391.7398        391.4931        390.8020        390.0906 
       DRXTP183        DRD320GW        DRXTZINC        DRXTP204        DRXTMAGN 
       388.7217        387.1669        385.2052        384.5863        376.6476 
       DRXTS120          cc_bmi        DRXTPHOS        DRXTP182        DRXTM181 
       372.3564        371.5824        371.5417        367.0030        359.1089 
       DRXTPFAT        DRXTS180        DRXTMFAT        DRXTS140        DRXTS040 
       358.4230        350.0913        346.2940        337.7077        335.9654 
       DRXTTFAT        DRXTS080        DRXTS160        DRXTS100        DRXTSFAT 
       334.0258        332.2106        329.4447        323.4145        314.9874 
       DRXTS060        DRXTM221        DRXTP226        DRXTP225        DRXTP205 
       309.6643        298.6327        291.5764        278.2607        270.4904 
       DRXTALCO        DRXTTHEO        svy_year        DRXTP184     demo_gender 
       265.6462        252.5937        238.4051        177.4844        173.3414 
      demo_race demo_race_black   demo_pregnant 
       138.7429        112.8859          0.0000

### Hybrid

In [74]:
# Step 1: Reduce the majority class (Under-sampling)
df1_majority <- df1 %>% filter(hyper == 0) %>% sample_n(nrow(df1_minority) * 1.5, replace = FALSE)

# Step 2: Increase the minority class (Over-sampling)
df1_minority_oversampled <- df1_minority %>% sample_n(nrow(df1_majority), replace = TRUE)

# Combine both
df1_balanced_hybrid <- bind_rows(df1_majority, df1_minority_oversampled)

# Check new class distribution
table(df1_balanced_hybrid$hyper)



   0    1 
5520 5520 

In [75]:
# Change here for different resample method
df <- df1_balanced_hybrid


# Split into training (70%) and testing (30%) sets
set.seed(24)
train_index <- createDataPartition(df$hyper, p = 0.7, list = FALSE)
train_data <- df[train_index, ]
test_data <- df[-train_index, ]

train_data$hyper <- as.factor(train_data$hyper)
test_data$hyper <- as.factor(test_data$hyper)

In [79]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data, 
                   num.trees = 1500, 
                   mtry = floor(sqrt(ncol(train_data) - 1) * 1.5), 
                   importance = "impurity",
                   probability = TRUE,
                   num.threads = parallel::detectCores())


# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# # Convert probabilities to class labels
# predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
# predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# # Model Evaluation
# conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# # Print Accuracy, Sensitivity, Specificity
# print(conf_matrix)

threshold <- 0.45  # Lower threshold slightly to increase sensitivity

predicted_class <- ifelse(predictions[,2] > threshold, "1", "0")  
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)
print(conf_matrix)


# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 1181  158
         1  475 1498
                                          
               Accuracy : 0.8089          
                 95% CI : (0.7951, 0.8221)
    No Information Rate : 0.5             
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.6178          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.7132          
            Specificity : 0.9046          
         Pos Pred Value : 0.8820          
         Neg Pred Value : 0.7592          
             Prevalence : 0.5000          
         Detection Rate : 0.3566          
   Detection Prevalence : 0.4043          
      Balanced Accuracy : 0.8089          
                                          
       'Positive' Class : 0               
   

demo_age_years    demo_age_cat        DRXTMOIS        DRXTCARB        DRXTFOLA 
      567.91731       367.82092        76.89415        69.69967        64.23250 
        DRXTVB6        DRDTSODI          DRXTVC        DRXTM201        DRXTIRON 
       64.18101        63.54745        62.80616        62.20194        61.77768 
       DRXTSELE        DRXTP204          cc_bmi        DRXTPROT        DRXTCHOL 
       61.63773        61.57586        60.61577        59.99718        59.73317 
       DRXTM161        DRXTPOTA        DRD320GW        DRXTVB12        DRXTCAFF 
       59.50475        58.58168        57.65271        57.64554        57.23187 
       DRXTP183        DRXTMAGN        DRXTCALC        DRXTZINC        DRXTFIBE 
       56.23965        56.21602        56.07804        55.49614        55.09849 
       DRXTCOPP        DRXTS120        DRXTP182        DRXTPHOS        DRXTPFAT 
       54.70640        51.53961        51.31831        51.27007        50.64821 
       DRXTM181        DRXTS180        DRXTMFAT        DRXTM221        DRXTS140 
       49.83063        48.99272        48.40162        48.26388        47.53258 
       DRXTS040        DRXTS080        DRXTS160        DRXTTFAT        DRXTS100 
       46.02303        45.69740        45.43275        45.04064        44.78874 
       DRXTS060        DRXTSFAT        DRXTALCO        DRXTP226        DRXTP225 
       44.15706        44.13747        43.92293        42.67011        41.17067 
       DRXTP205        DRXTTHEO     demo_gender        svy_year        DRXTP184 
       39.71727        35.87030        33.94916        33.77622        27.31529 
      demo_race demo_race_black   demo_pregnant 
       18.80864        14.09338         0.00000

In [18]:
# Change here for different resample method
df <- df1


# Split into training (70%) and testing (30%) sets
set.seed(42)
train_index <- createDataPartition(df$hyper, p = 0.7, list = FALSE)
train_data <- df[train_index, ]
test_data <- df[-train_index, ]

train_data$hyper <- as.factor(train_data$hyper)
test_data$hyper <- as.factor(test_data$hyper)

### Desrease size of majority

In [19]:
table(train_data$hyper)

# Separate majority and minority classes
train_data_majority <- train_data %>% filter(hyper == 0)
train_data_minority <- train_data %>% filter(hyper == 1)

set.seed(42)
# Oversample minority class (duplicate existing data)
train_data_majority_oversampled <- train_data_majority %>% sample_n(nrow(train_data_minority), replace = F)

# Combine balanced dataset
train_data_decrease <- bind_rows(train_data_minority, train_data_majority_oversampled)

# Check new class distribution
table(train_data_decrease$hyper)


    0     1 
20859  2609 


   0    1 
2609 2609 

In [21]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data_decrease, 
                   num.trees = 1000, 
                   mtry = sqrt(ncol(train_data_decrease) - 1), 
                   importance = "impurity",
                   probability = TRUE,  # Enables probability prediction
                   num.threads = num_cores)

# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# Convert probabilities to class labels
predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# Model Evaluation
conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# Print Accuracy, Sensitivity, Specificity
print(conf_matrix)

# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 6286  269
         1 2700  802
                                          
               Accuracy : 0.7048          
                 95% CI : (0.6958, 0.7137)
    No Information Rate : 0.8935          
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0.2242          
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.6995          
            Specificity : 0.7488          
         Pos Pred Value : 0.9590          
         Neg Pred Value : 0.2290          
             Prevalence : 0.8935          
         Detection Rate : 0.6250          
   Detection Prevalence : 0.6518          
      Balanced Accuracy : 0.7242          
                                          
       'Positive' Class : 0               
   

demo_age_years    demo_age_cat        DRXTMOIS        DRXTCARB        DRDTSODI 
     361.655370      241.412302       48.555707       47.164753       42.306531 
       DRXTCALC        DRXTZINC          DRXTVC        DRXTFOLA        DRXTFIBE 
      40.510213       40.208131       40.188513       38.559534       38.109841 
       DRXTCAFF        DRXTPROT        DRXTCHOL        DRXTIRON        DRXTM201 
      38.042211       37.794340       37.780467       37.583088       37.536500 
        DRXTVB6        DRXTSELE        DRXTPOTA        DRXTCOPP        DRD320GW 
      37.411285       37.336018       37.013931       36.682720       36.549675 
       DRXTVB12        DRXTP204        DRXTMAGN        DRXTM161        DRXTP183 
      36.430765       36.292138       36.166399       35.971811       35.681210 
       DRXTPHOS        DRXTS120        DRXTP182        DRXTS180          cc_bmi 
      34.860094       34.555661       33.776546       33.551888       33.467761 
       DRXTMFAT        DRXTPFAT        DRXTM181        DRXTS160        DRXTS140 
      33.057005       32.989746       32.914679       32.508404       31.829616 
       DRXTS100        DRXTTFAT        DRXTSFAT        DRXTS040        DRXTS080 
      31.234231       31.186844       31.153343       30.810491       30.503006 
       DRXTS060        DRXTP226        DRXTP225        DRXTM221        DRXTP205 
      29.647211       29.509831       28.844674       28.800945       25.901290 
       svy_year        DRXTALCO        DRXTTHEO     demo_gender        DRXTP184 
      23.888494       23.330081       22.600804       18.281242       16.462710 
      demo_race demo_race_black   demo_pregnant 
      13.803342        7.587191        0.000000

### Increase size of minority (Bootstrap)

In [22]:
table(train_data$hyper)

# Separate majority and minority classes
train_data_majority <- train_data %>% filter(hyper == 0)
train_data_minority <- train_data %>% filter(hyper == 1)

set.seed(24)
# Oversample minority class (duplicate existing data)
train_data_minority_oversampled <- train_data_minority %>% sample_n(nrow(train_data_majority), replace = T)

# Combine balanced dataset
train_data_bootstrap <- bind_rows(train_data_majority, train_data_minority_oversampled)

# Check new class distribution
table(train_data_bootstrap$hyper)


    0     1 
20859  2609 


    0     1 
20859 20859 

In [23]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data_bootstrap, 
                   num.trees = 500, 
                   mtry = sqrt(ncol(train_data_bootstrap) - 1), 
                   importance = "impurity",
                   probability = TRUE,  # Enables probability prediction
                   num.threads = num_cores)

# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# Convert probabilities to class labels
predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# Model Evaluation
conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# Print Accuracy, Sensitivity, Specificity
print(conf_matrix)

# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 8860  961
         1  126  110
                                          
               Accuracy : 0.8919          
                 95% CI : (0.8857, 0.8979)
    No Information Rate : 0.8935          
    P-Value [Acc > NIR] : 0.7042          
                                          
                  Kappa : 0.1351          
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.9860          
            Specificity : 0.1027          
         Pos Pred Value : 0.9021          
         Neg Pred Value : 0.4661          
             Prevalence : 0.8935          
         Detection Rate : 0.8810          
   Detection Prevalence : 0.9765          
      Balanced Accuracy : 0.5443          
                                          
       'Positive' Class : 0               
   

demo_age_years    demo_age_cat        DRXTMOIS        DRXTCARB        DRDTSODI 
     3090.98318      1946.85794       454.11640       430.73752       382.37954 
         DRXTVC        DRXTCAFF        DRXTM201        DRXTFOLA        DRD320GW 
      377.27856       374.64419       371.59840       367.43855       358.96206 
       DRXTCALC        DRXTIRON        DRXTM161        DRXTSELE        DRXTFIBE 
      358.44254       354.97379       354.44109       354.43501       351.94754 
       DRXTVB12         DRXTVB6        DRXTPOTA        DRXTCHOL        DRXTP183 
      350.14456       350.13290       348.75954       345.36647       343.39816 
       DRXTCOPP        DRXTZINC        DRXTPHOS        DRXTP204        DRXTPROT 
      342.75835       341.68432       339.94815       339.03432       337.42184 
       DRXTP182        DRXTS120        DRXTMAGN        DRXTPFAT        DRXTM181 
      331.35486       331.33988       324.38721       321.91466       317.31252 
       DRXTS180        DRXTMFAT          cc_bmi        DRXTS140        DRXTS040 
      314.56203       310.83621       307.18957       301.95701       294.49135 
       DRXTS100        DRXTS080        DRXTTFAT        DRXTS160        DRXTSFAT 
      294.24201       293.31942       290.19777       286.57055       276.15351 
       DRXTS060        DRXTM221        DRXTP226        DRXTP225        DRXTP205 
      275.96052       270.52464       256.51427       250.67242       237.49248 
       DRXTTHEO        svy_year        DRXTALCO     demo_gender        DRXTP184 
      222.93793       214.53537       184.01579       176.91708       156.55667 
      demo_race demo_race_black   demo_pregnant 
      125.52950        75.78033         0.00000

### Hybrid

In [24]:
# Step 1: Reduce the majority class (Under-sampling)
train_data_majority <- train_data %>% filter(hyper == 0) %>% sample_n(nrow(train_data_minority) * 1.5, replace = FALSE)

# Step 2: Increase the minority class (Over-sampling)
train_data_minority_oversampled <- train_data_minority %>% sample_n(nrow(train_data_majority), replace = TRUE)

# Combine both
train_data_hybrid <- bind_rows(train_data_majority, train_data_minority_oversampled)

# Check new class distribution
table(train_data_hybrid$hyper)



   0    1 
3913 3913 

In [26]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data_hybrid, 
                   num.trees = 1500, 
                   mtry = floor(sqrt(ncol(train_data_hybrid) - 1) * 1.5), 
                   importance = "impurity",
                   probability = TRUE,
                   num.threads = parallel::detectCores())


# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

threshold <- 0.45  # Lower threshold slightly to increase sensitivity

predicted_class <- ifelse(predictions[,2] > threshold, "1", "0")  
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)
print(conf_matrix)


# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 6594  315
         1 2392  756
                                          
               Accuracy : 0.7308          
                 95% CI : (0.7221, 0.7395)
    No Information Rate : 0.8935          
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0.2371          
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.7338          
            Specificity : 0.7059          
         Pos Pred Value : 0.9544          
         Neg Pred Value : 0.2402          
             Prevalence : 0.8935          
         Detection Rate : 0.6557          
   Detection Prevalence : 0.6870          
      Balanced Accuracy : 0.7198          
                                          
       'Positive' Class : 0               
   

demo_age_years    demo_age_cat        DRXTMOIS          cc_bmi        DRXTCARB 
      581.30780       383.25384        83.63974        75.93847        72.14065 
       DRDTSODI        DRXTCAFF        DRXTFOLA          DRXTVC        DRXTCALC 
       68.39863        66.62045        65.71525        65.35976        64.75043 
       DRD320GW        DRXTM201        DRXTFIBE        DRXTM161        DRXTPOTA 
       63.97481        62.34645        61.72225        61.26500        60.57544 
       DRXTSELE        DRXTMAGN         DRXTVB6        DRXTZINC        DRXTIRON 
       59.79938        59.35134        59.34851        58.74036        58.73596 
       DRXTCOPP        DRXTVB12        DRXTCHOL        DRXTP204        DRXTPHOS 
       57.95620        57.56017        56.47993        56.00806        54.97198 
       DRXTP183        DRXTPROT        DRXTS120        DRXTM181        DRXTPFAT 
       54.92103        54.24099        53.89365        51.15191        51.11813 
       DRXTP182        DRXTMFAT        DRXTM221        DRXTS180        DRXTS140 
       51.03273        50.94286        48.76247        48.72556        48.70614 
       DRXTS080        DRXTS040        DRXTS100        DRXTTFAT        DRXTS160 
       46.57046        46.37861        45.96169        45.39814        44.54945 
       DRXTSFAT        DRXTP225        DRXTS060        DRXTP226        DRXTP205 
       43.64755        43.08124        42.99350        42.82198        39.62024 
       DRXTALCO        DRXTTHEO        svy_year     demo_gender        DRXTP184 
       38.17040        35.79474        33.34986        27.86561        25.31320 
      demo_race demo_race_black   demo_pregnant 
       22.24152        14.13609         0.00000